In [1]:
import pandas as pd
import numpy as np
import os
import datetime

from itertools import product

In [2]:
def trata_dados(df, ano, pop_name):
    df['ID_Municipio'] = df['Município'].str[:6] #coluna do codigo do municipio
    df['Município'] = df['Município'].str[7:] #coluna do nome do municipio
    df['ano'] = ano #coluna do ano dos dados

    df.columns = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total', 'ID_Municipio', 'ano']
        
    df['ID_Municipio'] = pd.to_numeric(df['ID_Municipio'])
        
    #nao sei explicar direito o que faz, mas deixa tudo no formato desejado
    df = df.melt(id_vars=['Município', 'ID_Municipio', 'ano'], var_name='faixa_etaria', value_name=pop_name)
    
    return df

In [3]:
def gerar_df(cols, csv_name, value_name):
    df = pd.read_csv(f"dados populacionais/{csv_name}.csv", encoding='latin1', sep=';')

    padrao = df.iloc[[-1]]
    padrao.columns = cols

    padrao = padrao.melt(id_vars=['Município'], var_name='faixa_etaria', value_name=value_name)
    padrao.drop(labels=['Município'], axis=1, inplace=True)
    
    return df, padrao

In [4]:
#gera o df com a populacao total padrao de 2010

cols = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total']

#gera o df com a populacao total padrao de 2010
df, padrao_t = gerar_df(cols, "dados total/2010total", "pop_pad_total")

#gera o df com a populacao masculina padrao de 2010
df, padrao_h = gerar_df(cols, "dados homem/homem2010", "pop_pad_homem")

#gera o df com a populacao feminina padrao de 2010
df, padrao_m = gerar_df(cols, "dados mulher/mulher2010", "pop_pad_mulher")


In [5]:
padrao_t

,faixa_etaria,pop_pad_total
0,0a4,1338553
1,5a9,1497792
2,10a14,1711110
3,15a19,1742648
4,20a24,1757058
5,25a29,1727867
6,30a34,1617371
7,35a39,1435712
8,40a44,1392455
9,45a49,1312183


In [6]:
def gera_df_e_remove_linha(csv_name, title):
    #populacao total
    df = pd.read_csv(f"dados populacionais/{csv_name}.csv", encoding='latin1', sep=';')
    df.drop(df.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

    df = trata_dados(df, 2010, title)
    return df

#populacao total
df = gera_df_e_remove_linha("dados total/2010total", "pop_total")

#populacao masculina
df_h = gera_df_e_remove_linha("dados homem/homem2010", "pop_homem")

#populacao feminina
df_m = gera_df_e_remove_linha("dados mulher/mulher2010", "pop_mulher")


In [7]:
def pegar_arquivos(df, pasta, csv_name, title):
    directory = os.fsencode(f"./dados populacionais/{pasta}/") #populacao total

    anos = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

    i = 0

    #cria o dataframe com populacao total
    for file in os.listdir(directory):
        filename = os.fsdecode(file)

        if filename != f'{csv_name}.csv':
            aux = pd.read_csv(f"./dados populacionais/{pasta}/" + filename, encoding='latin1', sep=";")
            aux.drop(aux.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

            aux = trata_dados(aux, anos[i], title)

            df = df.append(aux)

            i = i + 1 #avanca o index de anos
            
    return df
            
df = pegar_arquivos(df, "dados total", "2010total", "pop_total")
df_h = pegar_arquivos(df_h, "dados homem", "homem2010", "pop_homem")
df_m = pegar_arquivos(df_m, "dados mulher", "mulher2010", "pop_mulher")

In [8]:
def unir_dfs(df1, df2, padrao2):
    df1 = df1.merge(df2, how="inner")
    df1 = df1.merge(padrao2, how="inner")
    return df1

df = df.merge(padrao_t, how="inner")

df = unir_dfs(df, df_h, padrao_h)
df = unir_dfs(df, df_m, padrao_m)

df

,Município,ID_Municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher
0,Abadia dos Dourados,310010,2010,0a4,418,1338553,211,684629,207,653924
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924
2,Abre Campo,310030,2010,0a4,914,1338553,464,684629,450,653924
3,Acaiaca,310040,2010,0a4,279,1338553,135,684629,144,653924
4,Açucena,310050,2010,0a4,687,1338553,368,684629,319,653924
...,...,...,...,...,...,...,...,...,...,...
153535,Virginópolis,317180,2019,Total,10510,19957444,5180,9828254,5330,10129190
153536,Virgolândia,317190,2019,Total,5380,19957444,2622,9828254,2758,10129190
153537,Visconde do Rio Branco,317200,2019,Total,42564,19957444,21194,9828254,21370,10129190
153538,Volta Grande,317210,2019,Total,5252,19957444,2512,9828254,2740,10129190


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153540 entries, 0 to 153539
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Município       153540 non-null  object
 1   ID_Municipio    153540 non-null  int64 
 2   ano             153540 non-null  int64 
 3   faixa_etaria    153540 non-null  object
 4   pop_total       153540 non-null  int64 
 5   pop_pad_total   153540 non-null  int64 
 6   pop_homem       153540 non-null  int64 
 7   pop_pad_homem   153540 non-null  int64 
 8   pop_mulher      153540 non-null  int64 
 9   pop_pad_mulher  153540 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 12.9+ MB


In [10]:
#usado para obter os nomes de todos os agravos e seus subgrupos
agrav = pd.read_csv("Planilha para o R - SINAN - Planilha.csv")
agrav = agrav.drop(labels='Unnamed: 0', axis=1)
agrav = agrav.fillna(0)
agrav.columns = ['cidsize', 'cid', 'agravo', 'subgrupo']

#listas com o nomes das doenças, anos, e meses
nomes_agravo = np.concatenate((agrav['agravo'].unique(), agrav['subgrupo'].unique()[1:]), axis=None)
faixas = df['faixa_etaria'].unique()
faixas = faixas[:-1]
anos = [ano for ano in range(2010, 2020)]
meses = [mes for mes in range(1, 13)]

In [11]:
#grupos de cidades analisadas
cidades_selecionadas = ["Betim", 'Brumadinho','Curvelo','Esmeraldas','Florestal','Fortuna de Minas',
                        'Igarapé','Juatuba','Maravilhas','Mário Campos','Martinho Campos',
                        'Papagaios','Pará de Minas','Paraopeba','Pequi','Pompéu','São Joaquim de Bicas',
                        'São José da Varginha','Sarzedo']

cidades_limitrofes = ['Abaeté','Araçaí','Belo Horizonte','Belo Vale','Bom Despacho','Bonfim','Cachoeira da Prata',
                      'Caetanópolis','Capim Branco','Conceição do Pará','Contagem','Cordisburgo','Corinto',
                      'Dores do Indaiá','Felixlândia','Ibirité','Igaratinga','Inhaúma','Inimutaba','Itaúna',
                      'Itabirito','Itatiaiuçu','Leandro Ferreira','Mateus Leme','Matozinhos','Moeda',
                      'Morada Nova de Minas','Morro da Garça','Nova Lima','Onça de Pitangui','Paineiras',
                      'Pedro Leopoldo','Pitangui','Presidente Juscelino','Quartel Geral','Ribeirão das Neves',
                      'Rio Manso','Santana de Pirapama','Santo Hipólito','Sete Lagoas']


cidades_prox_limitrofes = ['Araújos','Augusto de Lima','Baldim','Biquinhas','Carmo do Cajuru','Cedro do Abaeté',
                           'Conceição do Mato Dentro','Confins','Congonhas','Congonhas do Norte','Crucilândia',
                           'Estrela do Indaiá','Funilândia','Gouveia','Itaguara','Jaboticatubas','Jeceaba',
                           'Jequitibá','Lagoa Santa','Lassance','Luz','Moema','Monjolos','Nova Serrana',
                           'Ouro Preto','Piedade dos Gerais','Prudente de Morais',
                           'Raposos','Rio Acima','Sabará','São Gonçalo do Abaeté','São Gonçalo do Pará',
                           'São Gotardo','São José da Lapa','Santa Bárbara','Santa Luzia','Santana do Riacho',
                           'Santo Antônio do Monte','Serra da Saudade','Tiros','Vespasiano']


#gera uma lista com o ID de todas as cidades a serem avaliadas
todas = cidades_selecionadas + cidades_limitrofes + cidades_prox_limitrofes

cidades = df[df['Município'].isin(todas)]['ID_Municipio'].unique()

In [12]:
#cria um dataframe vazio, que armazenará todas as permutações de municipio, ano, mês e agravo
df_size = len(list(product(cidades, faixas, anos, meses, nomes_agravo)))
df_index = [i for i in range(df_size)]

agregados = pd.DataFrame({'id_municipio': pd.Series([], dtype='float32'),
                            'faixa_etaria': pd.Series([], dtype='str'),
                            'ano': pd.Series([], dtype='float32'),
                            'mes': pd.Series([], dtype='float32'),
                            'agravo': pd.Series([], dtype='str')},
                            index = df_index)

agregados

,id_municipio,faixa_etaria,ano,mes,agravo
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2855995,NaN,NaN,NaN,NaN,NaN
2855996,NaN,NaN,NaN,NaN,NaN
2855997,NaN,NaN,NaN,NaN,NaN
2855998,NaN,NaN,NaN,NaN,NaN


In [13]:
k = 0

data = agregados.values

for row in product(cidades, faixas, anos, meses, nomes_agravo):
    data[k] = row

    k = k + 1

In [14]:
np.savetxt("agregados_sinan.csv", data, fmt='%s', header="ID_Municipio faixa_etaria ano mes agravo", comments='')

In [15]:
agregados = pd.read_csv("agregados_sinan.csv", sep=" ")
agregados

,ID_Municipio,faixa_etaria,ano,mes,agravo
0,310020,0a4,2010,1,esquisto
1,310020,0a4,2010,1,leishvisc
2,310020,0a4,2010,1,leishteg
3,310020,0a4,2010,1,tubresp
4,310020,0a4,2010,1,colera
...,...,...,...,...,...
2855995,317120,>=80anos,2019,12,intoxicacao
2855996,317120,>=80anos,2019,12,lepto
2855997,317120,>=80anos,2019,12,malaria
2855998,317120,>=80anos,2019,12,sindrome


In [16]:
sinan = pd.read_csv("sinan_agravos.csv", encoding="latin1")

sinan

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (2,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID_AGRAVO,DT_NOTIFIC,DT_NASC,NU_ANO,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_ESCOL_N,ID_MN_RESI,idade2,faixaetaria,ano,mes,subgrupo,semana,causa3,agravo,capitulo
0,A90,2013-01-03,1974-11-08,2013.0,2013-01-02,4038.0,F,9.0,310090,38.0,35a39,2013,1,NaN,1.0,A90,dengue,I
1,A90,2013-01-03,1997-07-19,2013.0,2013-01-03,4015.0,F,5.0,310090,15.0,15a19,2013,1,NaN,1.0,A90,dengue,I
2,A90,2013-01-05,1952-07-27,2013.0,2013-01-05,4060.0,F,9.0,314790,60.0,60a64,2013,1,NaN,1.0,A90,dengue,I
3,A90,2013-01-07,1954-06-09,2013.0,2012-12-31,4058.0,F,1.0,316930,58.0,55a59,2013,1,NaN,2.0,A90,dengue,I
4,A90,2013-01-07,1949-02-15,2013.0,2013-01-07,4063.0,F,9.0,310090,63.0,60a64,2013,1,NaN,2.0,A90,dengue,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219811,A169,2019-12-30,NaN,2019.0,2019-12-30,4054.0,M,1.0,310420,54.0,50a54,2019,12,NaN,NaN,A16,tubresp,I
1219812,A169,2019-12-30,NaN,2019.0,2019-05-25,4047.0,M,2.0,316370,47.0,45a49,2019,12,NaN,NaN,A16,tubresp,I
1219813,A169,2019-12-30,NaN,2019.0,2019-12-30,4015.0,M,NaN,317070,15.0,15a19,2019,12,NaN,NaN,A16,tubresp,I
1219814,A169,2019-12-30,NaN,2019.0,2019-12-30,4036.0,M,4.0,311840,36.0,35a39,2019,12,NaN,NaN,A16,tubresp,I


In [17]:
sinan.columns

Index(['ID_AGRAVO', 'DT_NOTIFIC', 'DT_NASC', 'NU_ANO', 'DT_SIN_PRI',
       'NU_IDADE_N', 'CS_SEXO', 'CS_ESCOL_N', 'ID_MN_RESI', 'idade2',
       'faixaetaria', 'ano', 'mes', 'subgrupo', 'semana', 'causa3', 'agravo',
       'capitulo'],
      dtype='object')

In [18]:
#notificacoes de agravos por faixa etaria municipio ano e mes
aux = sinan[['ID_MN_RESI', 'faixaetaria', 'ano', 'mes', 'agravo', 'DT_NOTIFIC']].groupby(['ID_MN_RESI', 'faixaetaria', 'ano', 'mes', 'agravo'], as_index=False).count()
aux.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'notificacoes_total']

#adiciona os agravos considerados como "subgrupos" na tabela
aux_sub = sinan[['ID_MN_RESI', 'faixaetaria', 'ano', 'mes', 'subgrupo', 'DT_NOTIFIC']].groupby(['ID_MN_RESI', 'faixaetaria', 'ano', 'mes', 'subgrupo'], as_index=False).count()
aux_sub.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'notificacoes_total']

aux = aux.append(aux_sub, ignore_index=True)
aux

,ID_Municipio,faixa_etaria,ano,mes,agravo,notificacoes_total
0,310000,0a4,2018,9,intoxicacao,1
1,310000,0a4,2018,10,intoxicacao,1
2,310000,0a4,2019,12,intoxicacao,1
3,310000,15a19,2017,3,intoxicacao,1
4,310000,15a19,2017,8,esquisto,1
...,...,...,...,...,...,...
198724,317220,50a54,2018,2,intoxicacao,1
198725,317220,55a59,2013,1,lepto,1
198726,317220,5a9,2013,4,homicidio,1
198727,317220,60a64,2013,7,homicidio,1


In [19]:
def gerar_notificacoes(sinan, sexo2, nome):
    sinan_aux = sinan[sinan['CS_SEXO'] == sexo2]

    aux = sinan_aux[['ID_MN_RESI', 'faixaetaria', 'ano', 'mes', 'agravo', 'DT_NOTIFIC']].groupby(['ID_MN_RESI', 'faixaetaria', 'ano', 'mes', 'agravo'], as_index=False).count()
    aux.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', f'notificacoes_{nome}']

    aux_sub = sinan_aux[['ID_MN_RESI', 'faixaetaria', 'ano', 'mes', 'subgrupo', 'DT_NOTIFIC']].groupby(['ID_MN_RESI', 'faixaetaria', 'ano', 'mes', 'subgrupo'], as_index=False).count()
    aux_sub.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', f'notificacoes_{nome}']

    aux = aux.append(aux_sub, ignore_index=True)
    
    return aux

In [20]:
aux_h = gerar_notificacoes(sinan, 'M', "homem")
aux_m = gerar_notificacoes(sinan, 'F', "mulher")

In [21]:
aux = aux.merge(aux_h, how='left')
aux = aux.merge(aux_m, how='left')

In [22]:
taxas = agregados.merge(aux, how='left')

taxas = taxas.fillna(0)
taxas = taxas.astype({'notificacoes_total': 'int32',
                        'notificacoes_homem': 'int32',
                        'notificacoes_mulher': 'int32'})
taxas

,ID_Municipio,faixa_etaria,ano,mes,agravo,notificacoes_total,notificacoes_homem,notificacoes_mulher
0,310020,0a4,2010,1,esquisto,0,0,0
1,310020,0a4,2010,1,leishvisc,0,0,0
2,310020,0a4,2010,1,leishteg,0,0,0
3,310020,0a4,2010,1,tubresp,0,0,0
4,310020,0a4,2010,1,colera,0,0,0
...,...,...,...,...,...,...,...,...
2855995,317120,>=80anos,2019,12,intoxicacao,0,0,0
2855996,317120,>=80anos,2019,12,lepto,0,0,0
2855997,317120,>=80anos,2019,12,malaria,0,0,0
2855998,317120,>=80anos,2019,12,sindrome,0,0,0


In [23]:
taxas = df.merge(taxas, how='inner')
taxas

,Município,ID_Municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,mes,agravo,notificacoes_total,notificacoes_homem,notificacoes_mulher
0,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,1,esquisto,0,0,0
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,1,leishvisc,0,0,0
2,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,1,leishteg,0,0,0
3,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,1,tubresp,0,0,0
4,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,1,colera,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855995,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,12,intoxicacao,0,0,0
2855996,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,12,lepto,0,0,0
2855997,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,12,malaria,0,0,0
2855998,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,12,sindrome,0,0,0


In [24]:
taxas['grupo'] = np.nan

taxas.loc[(taxas['Município'].isin(cidades_selecionadas)), 'grupo'] = 1
taxas.loc[(taxas['Município'].isin(cidades_limitrofes)), 'grupo'] = 2
taxas.loc[(taxas['Município'].isin(cidades_prox_limitrofes)), 'grupo'] = 3

taxas = taxas[~taxas['grupo'].isnull()]

In [25]:
taxas.columns

Index(['Município', 'ID_Municipio', 'ano', 'faixa_etaria', 'pop_total',
       'pop_pad_total', 'pop_homem', 'pop_pad_homem', 'pop_mulher',
       'pop_pad_mulher', 'mes', 'agravo', 'notificacoes_total',
       'notificacoes_homem', 'notificacoes_mulher', 'grupo'],
      dtype='object')

In [26]:
taxas.columns = ['municipio', 'id_municipio', 'ano', 'faixa_etaria', 'pop_total',
       'pop_pad_total', 'pop_homem', 'pop_pad_homem', 'pop_mulher',
       'pop_pad_mulher', 'mes', 'agravo', 'notificacoes_total',
       'notificacoes_homem', 'notificacoes_mulher', 'grupo']

taxas = taxas[taxas['ano'] >= 2013]

taxas

,municipio,id_municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,mes,agravo,notificacoes_total,notificacoes_homem,notificacoes_mulher,grupo
50400,Abaeté,310020,2013,0a4,1309,1338553,669,684629,640,653924,1,esquisto,0,0,0,2.0
50401,Abaeté,310020,2013,0a4,1309,1338553,669,684629,640,653924,1,leishvisc,0,0,0,2.0
50402,Abaeté,310020,2013,0a4,1309,1338553,669,684629,640,653924,1,leishteg,0,0,0,2.0
50403,Abaeté,310020,2013,0a4,1309,1338553,669,684629,640,653924,1,tubresp,0,0,0,2.0
50404,Abaeté,310020,2013,0a4,1309,1338553,669,684629,640,653924,1,colera,0,0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855995,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,12,intoxicacao,0,0,0,3.0
2855996,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,12,lepto,0,0,0,3.0
2855997,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,12,malaria,0,0,0,3.0
2855998,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,12,sindrome,0,0,0,3.0


In [27]:
taxas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1999200 entries, 50400 to 2855999
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   municipio            object 
 1   id_municipio         int64  
 2   ano                  int64  
 3   faixa_etaria         object 
 4   pop_total            int64  
 5   pop_pad_total        int64  
 6   pop_homem            int64  
 7   pop_pad_homem        int64  
 8   pop_mulher           int64  
 9   pop_pad_mulher       int64  
 10  mes                  int64  
 11  agravo               object 
 12  notificacoes_total   int32  
 13  notificacoes_homem   int32  
 14  notificacoes_mulher  int32  
 15  grupo                float64
dtypes: float64(1), int32(3), int64(9), object(3)
memory usage: 236.4+ MB


In [28]:
taxas.to_csv('pre_taxa_sinan.csv', index=False)